In [4]:
from src.code_gen.code_generator import CCodeGenerator
from src.lexer.hulk_lexer import HulkLexer
from src.parser.hulk_parser import HulkParser
from src.evaluation import evaluate_reverse_parse
from src.semantics.semantic_analysis_pipeline import semantic_analysis_pipeline

lexer = HulkLexer(True)
parser = HulkParser()

In [14]:
tokens, errors = lexer('''  
    protocol ABC {
        getc() : Number;
    }
                 
    protocol AZX {
        qwerty() : String;
    }

    type A (x, y) {
        p = y;
        r = x;
        getc() => 3;
        qwerty() => "abc";
    }
                 
    type B inherits A {
        p = y;
        getc() => 6 + base();
    }
                 
    type C (x, y, z) inherits B (x + y, y + z) {
                 
    }
                 
    type D inherits C {
        d = x^2;
        getc() => base() + 5;
    }
                 
    let a = new D(2, 3, 4) in print(a.getc());
    ''')

derivation, operations, errors = parser(tokens)
ast = evaluate_reverse_parse(derivation, operations, tokens)
ast, semantic_errors, context, scope = semantic_analysis_pipeline(ast, False)

generator = CCodeGenerator()
code = generator(ast, context)

with open("test.c", "w") as out:
    out.write(code)

In [15]:
import subprocess

subprocess.run(["gcc", "test.c", "-o", "test.exe"], shell=True)
subprocess.run(['start', 'cmd', '/k', 'test.exe'], shell=True)

CompletedProcess(args=['start', 'cmd', '/k', 'test.exe'], returncode=0)